___
# __Convolutional Neural Network from scratch__
### _Author: Aki Taniguchi_
### _Original date: 19/02/2020_
### _Last update: 25/02/2020_
___

## I. Setting environment
___

In [1]:
import sys
sys.path.append("C:\\Users\\tngch\\Python Code and AI\\Neural Network")

# Load libraries
import numpy as np
from Deep_Neural_Network import activation_function
from Deep_Neural_Network import compute_cost
from Deep_Neural_Network import backward_activation_function

In [2]:
X = np.random.rand(3, 32, 32, 10)
Y = np.random.choice(2, 10)

A = {}
A['0'] = X

In [3]:
# Setting up the hyper-parameters ("LeNet-5")
architecture = ['init', 'conv', 'maxpool', 'conv', 'avgpool', 'fc', 'fc']
activations = ['init', 'relu', 'none', 'relu', 'none', 'relu', 'sigmoid']
filter_size = [0, 5, 2, 5, 2, 1, 1]
nb_kernel = [3, 8, 8, 16, 16, 120, 84] # first is RGB, fully connected layers are the number of neurons
padding = [0, 0, 0, 0, 0, 0, 0]
stride = [0, 1, 2, 1, 2, 0, 0]
L = len(architecture)

## II. Initialize model
___

In [4]:
# Define Shape of Output A. Note that the layer with full connection needs to be vectorized, but won't be here
# dim A is (channel, height, width, observation)
# width and height are both determined: int[(x + 2p - f) / s + 1]
def get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A):

    layer_shape = [A['0'].shape]

    m = A['0'].shape[3]
    n_h = A['0'].shape[1]
    n_w = A['0'].shape[2]
    
    L = len(architecture)
    f = filter_size
    k = nb_kernel
    p = padding
    s = stride

    # Note the last layer has only 2 outcome as we are not doing a softmax. This needs to be changed once it will be implemented.
    for l in range(1, L+1):
        if l != L:
            if architecture[l] != 'fc':
                n_h = int((n_h + 2*p[l] - f[l]) / s[l] + 1)
                n_w = int((n_w + 2*p[l] - f[l]) / s[l] + 1)
                layer_shape.append((k[l], n_h, n_w, m))
            else:
                layer_shape.append((k[l], m))
        else:
            layer_shape.append((2, m))

    return layer_shape

In [5]:
# Test get_layer_shape
layer_shape = get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A)

layer_shape

[(3, 32, 32, 10),
 (8, 28, 28, 10),
 (8, 14, 14, 10),
 (16, 10, 10, 10),
 (16, 5, 5, 10),
 (120, 10),
 (84, 10),
 (2, 10)]

In [6]:
# Parameters size:
# If Conv/Pool: (curr channel, prev channel, filter, filter) and (cur channel, 1, 1, 1)
# If FC from Conv/Pool: (curr channel, vectorized[prev output])
# If FC from FC: (curr channel, prev channel)
# bias is always (curr channel, 1) when FC
def initialize_model(architecture, filter_size, nb_kernel, padding, stride, A):

    W = {}; b = {}; Z = {}; dA = {}; dZ = {}
    L = len(architecture)
    f = filter_size
    k = nb_kernel
    m = A['0'].shape[3]
    layer_shape = get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A)

    for l in range(1, L):

        # We need to initialize output as well to allocate the convolution output per index (otherwise Python doesn't allow allocation)
        Z[str(l)] = np.zeros((layer_shape[l]))
        A[str(l)] = np.zeros((layer_shape[l]))

        # Now initializing parameters
        if architecture[l] == 'conv':
            
            W[str(l)] = np.random.randn(k[l], k[l-1], f[l], f[l]) * 0.01
            b[str(l)] = np.zeros((k[l], 1, 1, 1))
            dZ[str(l)] = np.zeros((layer_shape[l]))
            dA[str(l)] = np.zeros((layer_shape[l]))

        # Note that there is no parameter, nor Z for Pooling layers
        elif (architecture[l] == 'maxpool') | (architecture[l] == 'avgpool'):
            W[str(l)] = 0
            b[str(l)] = 0
            Z[str(l)] = 0
            dZ[str(l)] = 0
            dA[str(l)] = np.zeros((layer_shape[l-1]))

            # We specifically need to raise error if pooling layer kernel is different from convolutional layer kernel
            if k[l] != k[l-1]:
                raise ValueError("Pooling layer kernel # needs to be equal to Convolutional layer kernel #")

        elif architecture[l] =='fc':
            # Parameters size different at the moment of change from conv to fc due to vectorized output
            if architecture[l-1] != 'fc':
                W[str(l)] = np.random.randn(k[l], int(np.prod(A[str(l-1)].shape) / m))
            else:
                W[str(l)] = np.random.randn(k[l], k[l-1])
            
            b[str(l)] = np.zeros((k[l], 1))
            dZ[str(l)] = np.zeros((layer_shape[l]))
            dA[str(l)] = np.zeros((layer_shape[l]))

    return A, Z, W, b, dA, dZ

In [7]:
# Test initialize_model
A, Z, W, b, dA, dZ = initialize_model(architecture, filter_size, nb_kernel, padding, stride, A)

for l in range(L):
    print(architecture[l])
    print("A[{0}] shape: {1}".format(l, A[str(l)].shape))
    if l != 0:
        if (architecture[l] == 'maxpool') | (architecture[l] == 'avgpool'):
            print("Z[{0}] shape: {1}".format(l, Z[str(l)]))
            print("W[{0}] shape: {1}".format(l, W[str(l)]))
            print("b[{0}] shape: {1}".format(l, b[str(l)]))
            print("dA[{0}] shape: {1}".format(l, dA[str(l)].shape))
            print("dZ[{0}] shape: {1}".format(l, dZ[str(l)]))              
        else:
            print("Z[{0}] shape: {1}".format(l, Z[str(l)].shape))
            print("W[{0}] shape: {1}".format(l, W[str(l)].shape))
            print("b[{0}] shape: {1}".format(l, b[str(l)].shape))
            print("dA[{0}] shape: {1}".format(l, dA[str(l)].shape))
            print("dZ[{0}] shape: {1}".format(l, dZ[str(l)].shape))

init
A[0] shape: (3, 32, 32, 10)
conv
A[1] shape: (8, 28, 28, 10)
Z[1] shape: (8, 28, 28, 10)
W[1] shape: (8, 3, 5, 5)
b[1] shape: (8, 1, 1, 1)
dA[1] shape: (8, 28, 28, 10)
dZ[1] shape: (8, 28, 28, 10)
maxpool
A[2] shape: (8, 14, 14, 10)
Z[2] shape: 0
W[2] shape: 0
b[2] shape: 0
dA[2] shape: (8, 28, 28, 10)
dZ[2] shape: 0
conv
A[3] shape: (16, 10, 10, 10)
Z[3] shape: (16, 10, 10, 10)
W[3] shape: (16, 8, 5, 5)
b[3] shape: (16, 1, 1, 1)
dA[3] shape: (16, 10, 10, 10)
dZ[3] shape: (16, 10, 10, 10)
avgpool
A[4] shape: (16, 5, 5, 10)
Z[4] shape: 0
W[4] shape: 0
b[4] shape: 0
dA[4] shape: (16, 10, 10, 10)
dZ[4] shape: 0
fc
A[5] shape: (120, 10)
Z[5] shape: (120, 10)
W[5] shape: (120, 400)
b[5] shape: (120, 1)
dA[5] shape: (120, 10)
dZ[5] shape: (120, 10)
fc
A[6] shape: (84, 10)
Z[6] shape: (84, 10)
W[6] shape: (84, 120)
b[6] shape: (84, 1)
dA[6] shape: (84, 10)
dZ[6] shape: (84, 10)


## III. Creating helper functions
___
Padding  
Slicing  
Vectorization

In [8]:
# We won't be using numpy's pad function as this one is enough and quick to operate
# Only works for 3D matrix
def add_padding(A, padding, value=0, axis=(0, 1, 2)):

    kernel, height, width = axis

    horizontal_pad = np.zeros((A.shape[kernel], padding, A.shape[width] + 2*padding)) + value
    vertical_pad = np.zeros((A.shape[kernel], A.shape[height], padding)) + value

    padded_A = np.concatenate((vertical_pad, A), axis=width)
    padded_A = np.concatenate((padded_A, vertical_pad), axis=width)
    padded_A = np.concatenate((horizontal_pad, padded_A), axis=height)
    padded_A = np.concatenate((padded_A, horizontal_pad), axis=height)

    return padded_A

In [9]:
# Test add_padding
pad = 1
padded_matrix = add_padding(A['0'][:,:,:,0], pad, value=0, axis=(0, 1, 2))

print("Padding:", pad)
print("Original matrix shape:", A['0'][:,:,:,0].shape)
print("Padded matrix:", padded_matrix.shape)
print("___________________")
print("Original matrix:", A['0'][:,:,:,0])
print("___________________")
print("Padded matrix:", padded_matrix)

Padding: 1
Original matrix shape: (3, 32, 32)
Padded matrix: (3, 34, 34)
___________________
Original matrix: [[[0.80910971 0.87126192 0.9479845  ... 0.49261322 0.74834998 0.88788139]
  [0.50733089 0.9281072  0.40057399 ... 0.84011627 0.54032842 0.50940143]
  [0.57508981 0.86727363 0.80406552 ... 0.60838744 0.39553176 0.97301442]
  ...
  [0.30617513 0.06138716 0.38945002 ... 0.93768146 0.07970471 0.07837573]
  [0.26788262 0.43128458 0.00665561 ... 0.56605573 0.48859255 0.88772732]
  [0.08624652 0.37492544 0.73017502 ... 0.32011413 0.25328062 0.45150791]]

 [[0.1372327  0.68208641 0.66545846 ... 0.19449202 0.32870502 0.19413085]
  [0.847629   0.13602974 0.86673516 ... 0.10573889 0.66740911 0.16302562]
  [0.25002816 0.388352   0.93204938 ... 0.16045953 0.1397784  0.91925739]
  ...
  [0.05903123 0.56758681 0.38333621 ... 0.96678757 0.49682607 0.05100984]
  [0.8637013  0.39219656 0.64871522 ... 0.54660719 0.53349473 0.45051365]
  [0.72738251 0.70959769 0.54415291 ... 0.10348158 0.93052842 

In [10]:
# Note this only works for a 4D matrix
def slicing(A, layer, architecture, filter_size, padding, stride, side='fwd'):

    if side == 'fwd':
        ref = layer - 1
    elif side == 'back':
        ref = layer
    # Padding the matrix to work the slicing on
    # This is base matrix where we are going to create all the necessary slicing (hence the need to pad it first)
    # Also note that we don't need to specify the value of kernel, as we won't slice through the depth (instead we take it all)
    # It takes as input a 4D matrix (for all observation), but return slice for only one observation
    f = filter_size
    s = stride
    kernel, height, width, observation = A[str(layer)].shape

    for m in range(observation):
        padded_A = add_padding(A[str(ref)][:, :, :, m], padding[layer], value=0, axis=(0, 1, 2))
            
        for i in range(height):
            v1 = i * s[layer]
            v2 = i * s[layer] + f[layer]

            for j in range(width):
                h1 = j * s[layer]
                h2 = j * s[layer] + f[layer]

                # If convolutional layer, we need to return a fxfxkernel 3D matrix
                if architecture[layer] == 'conv':
                    slice = padded_A[:, v1:v2, h1:h2]
                    # We don't exactly need k, but we still return it to yield the exact number of output in both cases
                    k = 0

                    yield slice, i, j, k, m

                # If pooling layer, we need an output slice of a fxf 2D matrix for each l-1 kernel (we have set kernel to be (l) given that (l-1) = (l) kernel)
                elif (architecture[layer] == 'maxpool') | (architecture[layer] == 'avgpool'):
                    for k in range(kernel):
                        slice = padded_A[k, v1:v2, h1:h2]

                        yield slice, i, j, k, m

In [11]:
# Test slicing
layer = 2; cnt = 0; dim_check = 0

for slice, i, j, k, m in slicing(A, layer, architecture, filter_size, padding, stride, 'fwd'):
    if (i < 2) & (j < 2) & (k==0) & (m==0):
        print(i, j, k, m, slice.shape, slice)
    cnt += 1

if architecture[layer] == 'conv':
    dim_check = np.prod(A[str(layer)].shape) / A[str(layer)].shape[0]
    theoretical_shape = (nb_kernel[layer-1], filter_size[layer], filter_size[layer])
elif (architecture[layer] == 'maxpool') | (architecture[layer] == 'avgpool'):
    dim_check = np.prod(A[str(layer)].shape)
    theoretical_shape = (filter_size[layer], filter_size[layer])

print("\n___________________")
print("Current layer architecture:", architecture[layer])
print("# of slices:", cnt)
print("Theoretical # of slices:", dim_check)
print("Theoretical dimension of slice:", theoretical_shape)
print("Real dimenion of slice:", slice.shape)

0 0 0 0 (2, 2) [[0. 0.]
 [0. 0.]]
0 1 0 0 (2, 2) [[0. 0.]
 [0. 0.]]
1 0 0 0 (2, 2) [[0. 0.]
 [0. 0.]]
1 1 0 0 (2, 2) [[0. 0.]
 [0. 0.]]

___________________
Current layer architecture: maxpool
# of slices: 15680
Theoretical # of slices: 15680
Theoretical dimension of slice: (2, 2)
Real dimenion of slice: (2, 2)


In [12]:
# Helper function for vectorization / devectorization
def vectorization(X, architecture, layer, type='forward'):

    # Vectorization
    if type == 'forward':
        if (architecture[layer] == 'fc') & (architecture[layer-1] != 'fc'):
            # When the layer # is 'Fully Connected', we need to vectorize the previous output
            # or pass the previous output if it is not 'fc' as no need to do any vectorization
            # This does not work for Z, W, or b with the very first hidden layer (only A) as they do not have any value assigned 
            dim_1, dim_2, dim_3, dim_4 = X[str(layer-1)].shape
            new_X = X[str(layer-1)].reshape(dim_1*dim_2*dim_3, dim_4)
        else:
            new_X = X[str(layer-1)]
    
    # Devectorization
    elif type == 'inverted':
        if (architecture[layer+1] == 'fc') & (architecture[layer] == 'maxpool') | (architecture[layer] == 'avgpool'):
            new_X = X[str(layer+1)].reshape(X[str(layer-1)].shape)
        elif (architecture[layer+1] == 'fc') & (architecture[layer] == 'conv'):
            new_X = X[str(layer+1)].reshape(X[str(layer)].shape)
        else:
            new_X = X[str(layer)]
    
    else:
        raise ValueError("Type unknown: must be 'forward' or 'inverted'")

    return new_X

In [13]:
# Test vectorization function
for l in range(1, L):
    print("Layer:", l, architecture[l])
    print(A[str(l)].shape)

print("\n_____________")
print(architecture[1], architecture[2])
print("Vectorized shape:", vectorization(A, architecture, 2, 'forward').shape)
print("Expected shape:", A['1'].shape)

print("\n_____________")
print(architecture[2], architecture[3])
print("Devectorized shape:", vectorization(A, architecture, 3, 'inverted').shape)
print("Expected shape:", A['3'].shape)

print("\n_____________")
print(architecture[4], architecture[5])
print("Vectorized shape:", vectorization(A, architecture, 5, 'forward').shape)
print("Expected shape:", (A['4'].shape[0]*A['4'].shape[1]*A['4'].shape[2], A['4'].shape[3]))

print("\n_____________")
print(architecture[4], architecture[5])
print("Devectorized shape:", vectorization(A, architecture, 5, 'inverted').shape)
print("Expected shape:", A['4'].shape)

Layer: 1 conv
(8, 28, 28, 10)
Layer: 2 maxpool
(8, 14, 14, 10)
Layer: 3 conv
(16, 10, 10, 10)
Layer: 4 avgpool
(16, 5, 5, 10)
Layer: 5 fc
(120, 10)
Layer: 6 fc
(84, 10)

_____________
conv maxpool
Vectorized shape: (8, 28, 28, 10)
Expected shape: (8, 28, 28, 10)

_____________
maxpool conv
Devectorized shape: (16, 10, 10, 10)
Expected shape: (16, 10, 10, 10)

_____________
avgpool fc
Vectorized shape: (400, 10)
Expected shape: (400, 10)

_____________
avgpool fc
Devectorized shape: (120, 10)
Expected shape: (16, 5, 5, 10)


## IV. Convolutional Forward Propagation
___
Convolutional Layer:
$$ Z ^{[l]} _{(c, i, j, m)} = \sum _{k} ^{K} \sum _{h=1} ^{f^{[l]}_h} \sum_{w=1} ^{f^{[l]}_w} W^{[l]} _{(c, k, h, w)} \times \operatorname{pad} (A^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w)}, m) + b^{[l]} _{(c)} \tag{1} $$  
$$ A^{[l]} _{(c, i, j, m)} = g(Z ^{[l]} _{(c, i, j, m)}) \tag{2} $$  
With: $i = h^{[l]} = \lfloor \frac {h^{[l-1]} + 2p^{[l]} - f^{[l]} _h} {s^{[l]}} \rfloor + 1 $, $j = w^{[l]} = \lfloor \frac {w^{[l-1]} + 2p^{[l]} - f^{[l]} _w} {s^{[l]}} \rfloor + 1 $  

Here, $h^{[l]}$ and $w^{[l]}$ (therefore i and j) are the height and width of layer \[l], $p^{[l]}$ is the padding added to the output \[l-1], $s^{[l]}$ is the stride layer l, c is the channel of layer \[l], K is the channel of layer \[l-1], $f^{[l]}_h$ and $f^{[l]}_w$ the height and width of the filter of layer \[l], and lastly m the number of observation.    
Example, i = 1, j = 2...

Pooling is to reduce size, and makes the computation easier. There is also an L2 pooling in order to control overfitting.  
Max Pooling Layer:  
$$ A ^{[l]} _{(c, i, j, m)} = \max {( A^{[l-1]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)} )} \tag{3} $$  
Average Pooling Layer:  
$$ A ^{[l]} _{(c, i, j, m)} = \frac {1} {f^{[l]}_h \times f^{[l]}_w} \times \sum _{h=1} ^{f^{[l]}_h} \sum_{w=1} ^{f^{[l]}_w} A^{[l-1]} _{(s(i-1)+h, s(j-1)+w, m)} \tag{4} $$    

With: $h_{1} = s(i-1) + 1$, $h_{2} = s(i-1) + f^{[l]}_h$, $w_{1} = s(j-1) + 1$, $w_{2} = s(j-1) + f^{[l]}_w$, i and j is calculated the same way as convolutional layer. In this layer, there is no $Z^{[l]}$, and the number of channel is equivalent to the previous layer.  

Vectorization to Fully connected Layer:  
In order to pass the output to the fully connected layer, we need to reduce the dimension e.g. vectorize the output so that it becomes from size (k, h, w, m) to (k*h*w, m), 4D ==> 2D. We will call the vectorization function as f, such as:  
$$ f(A^{[l]} _{(k^{[l]}, h^{[l]}, w^{[l]}, m)}) = A^{[l]} _{(k^{[l]} \times h^{[l]} \times w^{[l]}, m)} \tag{5} $$  

Fully connected Layer:  
Once the vectorization is done, we can then compute the output of the fully connected layer (e.g. traditional Neural Network calculation):  
$$Z^{[l]} = W^{[l]} \times f(A^{[l-1]}) + b^{[l]} \tag{6} $$
$$A^{[l]} = g(Z^{[l]}) \tag{7} $$

In [14]:
def convolutional_forward_propagation(A, Z, architecture, activations, filter_size, nb_kernel, padding, stride):
    
    L = len(architecture)

    # Through each layer
    for l in range(1, L):
            
        # We operate the convolution nb kernel time (corresponding to the output layer kernel, or the current layer filter kernel size)
        if architecture[l] == 'conv':
            # As mentionned in slicing, loop over kernel is different for pooling and conv (pooling is already in slicing, vs conv is looping output kernel size time)
            for k in range(A[str(l)].shape[0]):
                for slice, i, j, _, m in slicing(A, l, architecture, filter_size, padding, stride, 'fwd'):
                    # We can see that the formula is highly similar to the usual forward propagation, the only diff is that it is an element-wise
                    # multiplication instead of a matrix multiplication, and we should return a scalar only
                    Z[str(l)][k, i, j, m] = np.sum(np.multiply(W[str(l)][k], slice) + b[str(l)][k])
                    A[str(l)][k, i, j, m] = activation_function(activations[l], Z[str(l)][k, i, j, m])

        elif architecture[l] == 'maxpool':
            for slice, i, j, k, m in slicing(A, l, architecture, filter_size, padding, stride, 'fwd'):
                A[str(l)][k, i, j, m] = np.max(slice)

        elif architecture[l] == 'avgpool':
            for slice, i, j, k, m in slicing(A, l, architecture, filter_size, padding, stride, 'fwd'):
                A[str(l)][k, i, j, m] = np.mean(slice)

        elif architecture[l] == 'fc':
            Z[str(l)] = np.dot(W[str(l)], vectorization(A, architecture, l, type='forward')) + b[str(l)]
            A[str(l)] = activation_function(activations[l], Z[str(l)])

    return Z, A

In [15]:
# Test forward prop
Z, A = convolutional_forward_propagation(A, Z, architecture, activations, filter_size, nb_kernel, padding, stride)

for l in range(L):
    print("Layer {0}, Shape {1}".format(l, A[str(l)].shape))

print("Last layer value:", A[str(L-1)])
print("Cost:", compute_cost(A[str(L-1)], Y))

Layer 0, Shape (3, 32, 32, 10)
Layer 1, Shape (8, 28, 28, 10)
Layer 2, Shape (8, 14, 14, 10)
Layer 3, Shape (16, 10, 10, 10)
Layer 4, Shape (16, 5, 5, 10)
Layer 5, Shape (120, 10)
Layer 6, Shape (84, 10)
Last layer value: [[0.46667822 0.47875382 0.5122367  0.44255163 0.481632   0.48072479
  0.46234001 0.47405187 0.46649445 0.45243979]
 [0.54567069 0.53869714 0.52471174 0.61638047 0.52244765 0.56533697
  0.56989171 0.58758283 0.5363253  0.5815607 ]
 [0.45981101 0.44111496 0.43210486 0.45670838 0.44044764 0.44088354
  0.44108746 0.45655059 0.45829139 0.42018165]
 [0.62575553 0.62493445 0.65338135 0.58879335 0.61693255 0.63863689
  0.6437032  0.59910572 0.62176423 0.6301343 ]
 [0.49032712 0.51932903 0.52331193 0.50839228 0.53384832 0.45934043
  0.54134594 0.52826419 0.52438411 0.51228038]
 [0.39589493 0.37729544 0.4002846  0.43858137 0.41653248 0.41374909
  0.37677974 0.3827707  0.38849233 0.3790446 ]
 [0.34007048 0.32775338 0.32862267 0.3493407  0.34374731 0.33459917
  0.3191107  0.36800

## V. Convolutional Backward Propagation
___
Recall the generalized scheme: Conv ==> Pool ==> Vectorized ==> FC.  
Derivation is therefore: FC ==> Devectorized ==> Pool ==> Conv.
  
Fully connected layer:  
We simply reuse the traditional fully connected neural network formulas:
$$dA^{[L]} = \frac{\partial Cost}{\partial A^{[L]}} = \frac{-Y}{A^{[L]}} + \frac{(1-Y)}{(1-A^{[L]})} \tag{8}$$
$$dZ^{[L]} = \frac{\partial Cost}{\partial A^{[L]}} \frac{\partial A^{[L]}}{\partial Z^{[L]}} = dA^{[L]} * g'(Z^{[L]}) \tag{9}$$

If there would have only one fully connected layer, we would stop at equation (9) and get gradients for parameters W and b.
$$dW^{[l]} = \frac{1}{m} dZ^{[l]} \times A'^{[l-1]} \tag{10}$$  
$$db^{[l]} = \frac{1}{m} \sum \limits_{i=1}^{m} {dZ^{[l]}} \tag{11}$$  

For more fully connected layers at the end, it goes:
$$dA^{[l-1]} = W'^{[l]} \times dZ^{[l]} \tag{10}$$
$$dZ^{[l-1]} = W'^{[l]} \times dZ^{[l]} * g'(Z^{[l-1]}) \tag{11}$$  
And we use (10) and (11) for dW, db of layer \[l-1].  
  
  
Devectorization:  
Recall that the first fully connected layer input is the vectorized \[l-1] output, e.g. assuming the vectorization is done at layer \[l]: $Z^{[l]} = W^{[l]} \times f(A^{[l-1]}) + b^{[l]}$. This means that we need to calculate $\frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial f(A^{[l-1]})}$ first.  
$$ dA^{[l-1]} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial f(A^{[l-1]})} = W'^{[l]} \times dZ^{[l]}$$  
And now we can devectorize:  
$$ \operatorname{devectorized} (dA^{[l-1]}) = F^{-1}(\frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial f(A^{[l-1]})}) = F^{-1}(W'^{[l]} \times dZ^{[l]}) \tag{12} $$
Which is of dimension $(k^{[l-1]}, h^{[l-1]}, w^{[l-1]}, m)$ if there is no pooling layer, and $(k^{[l-2]}, h^{[l-2]}, w^{[l-2]}, m)$ if there is a pooling layer (the forward propagation of pooling is done to reduce the size of the matrix, but the backprop returns 0 or average in the original convolution layer matrix, and therefore we never really uses the pooling dimension, which keeps the size of the input for the output as we will see in the next section). We can now pass on the devectorized gradient to the following layers.
  
Pooling layer:  
Pooling layers have no parameters, but we still need to instruct the following layers of the "winner cells" to continue the backward propagation process. In each case, we have: $h_{1} = s(i-1) + 1$, $h_{2} = s(i-1) + f^{[l]}_h$, $w_{1} = s(j-1) + 1$, $w_{2} = s(j-1) + f^{[l]}_w$, i and j is calculated the same way as convolutional layer, and dimension of the output is the same as the dimension of the input.  

Max Pooling:
For Max pooling, winner cells are the one which are the maximum. The rest are equals to 0. Therefore:  
$$ dA^{[l]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)} = \frac{\partial Cost}{\partial f(A^{[l]})} \frac{\partial f(A^{[l]})}{\partial A^{[l]}} = \operatorname{arg\,max} (\operatorname{devectorized} (dA^{[l]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)}) ) \tag{13} $$

Average Pooling:
Reversing the average means we allocate back to each cell the value of the average. Therefore each cell receive the value of $\frac {average} {\#cells}$, i.e.:
$$ dA^{[l]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)} = \frac{\partial Cost}{\partial f(A^{[l]})} \frac{\partial f(A^{[l]})}{\partial A^{[l]}} = \frac {1} {f^{[l]}_h \times f^{[l]}_w} \times \operatorname{devectorized} (dA^{[l]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)}) \tag{14} $$  
  
Convolutional layer:  
We can now continue the derivation following the chain rule:  
$$ dZ^{[l]} = \frac{\partial Cost}{\partial A^{[l]}} \frac{\partial A^{[l]}} {\partial Z^{[l]}} = dA^{[l]} * g'(Z^{[l]}) $$
With $ dA^{[l]} $ being the pooling gradient matrix, of dimension $A^{[l]conv]}$ given we use the Convolutional layer output as input for pooling backpropagation.
Given equation (1), it is easy to derive dW as: 
$$ dW^{[l]} _{(c, k, h, w)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial W^{[l]} _{(k, h, w)}} = \sum _{i=1} ^{n^{[l]}_H} \sum_{j=1} ^ {n^{[l]}_W} dZ ^{[l]} _{(c, i, j, m)} \times \operatorname{pad} (A^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w, m)}) \tag{15}$$

Indeed, for a specific filter c, the value of the kth kernel ith row and jth column of the gradients of W is a specific value of the previous output, that we have many times because of the parameter sharing specificity of convolution. We do use this specific value of $W^{[l]}_{(c, k, i, j)}$ $n^{[l]}_H \times n^{[l]}_W$ times, and this for each $dZ^{[l]} _{(c, i, j, m)}$. Note also that $k \in [1, K^{[l]}]$, $i \in [1, f^{[l]}_h]$, $j \in [1, f^{[l]}_w]$.  
The same reasoning can be applied to compute $db^{[l]}$, which is given by:
$$ db^{[l]} _{(c)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial b^{[l]} _{(c)}} = \sum _{i=1} ^{n^{[l]}_H} \sum_{j=1} ^ {n^{[l]}_W} dZ ^{[l]} _{(c, i, j, m)} \tag{16}$$

Note that if the next layer is also a convolutional layer, e.g. an architecture of conv -> conv, the calculation of $dA^{[l]}$ becomes:
$$ dA^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w, m)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial A^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w, m)}} = \sum_{c=1} ^ {n^{[l]}_C} \sum _{i=1} ^{n^{[l]}_H} \sum_{j=1} ^ {n^{[l]}_W}  W^{[l]} _{(c, k, h, w)} \times dZ ^{[l]} _{(c, i, j, m)}$$  

Simplifying the indexing to: $dA^{[l-1]} _{(k, i', j', m)}$, which means $i' = s(i-1) + h$ and $j' = s(i-1) + w$, we find that $i = \frac {i'-h} {s} + 1$, $j = \frac {j'-w} {s} + 1$. However, i and j cannot be below 1 as they correspond to the minimum index of the output matrix $dZ^{[l]}$, and it also cannot go beyond its size. In other words, i and j are both floored at 1 and are capped at $n^{[l]}_H$ and $n^{[l]}_W$ respectively. Therefore, $\forall h \in [1, f^{[l]}_h]$, $\forall w \in [1, f^{[l]}_w]$:  
$$ dA^{[l-1]} _{(k, i', j', m)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial A^{[l-1]} _{(k, i', j', m)}} = \sum_{c=1} ^ {n^{[l]}_C} \sum _{i=\max (1, \frac {i'-h} {s} + 1)} ^{\min (i', n^{[l]}_H)} \sum_{j= \max (1, \frac {j'-w} {s} + 1)} ^ {\min (j', n^{[l]}_W)} W^{[l]} _{(c, k, h, w)} \times dZ ^{[l]} _{(c, i, j, m)} \tag{17} $$  

We finally unpad $dA^{[l-1]} _{(k, i', j', m)}$ by the appropriate padding value to come back to the original input matrix size, so that we can carry calculating gradients through (14), (15) and (16).

In [ ]:
def convolutional_backward_propagation(A, Z, dA, dZ, architecture, filter_size, padding, stride):

    # Works similar to normal backprop for fully connected layer
    # We need to compute the derivative of the last layer separately, as it is a non-generic formula
    # 1e-8 is added for numeric stability

    L = len(architecture)-1; dW = {}; db = {}
    s = stride
    f = filter_size
    m = A['0'].shape[3]

    dAL = -Y / (A[str(L)] + 1e-8) + (1-Y) / ((1-A[str(L)]) + 1e-8)
    dZ[str(L)] = dAL * backward_activation_function(activations[L], Z[str(L)])
    dW[str(L)] = 1/m * np.dot(dZ[str(L)], A[str(L-1)].T)
    db[str(L)] = 1/m * np.sum(dZ[str(L)], axis=1, keepdims=True)

    # We can now compute the generalized backward propagation
    for l in range(L-1, 0, -1):

        # We use the same code for fully connected layer
        if architecture[l] == 'fc':
            dZ[str(l)] = np.dot(W[str(l+1)].T, dZ[str(l+1)]) * backward_activation_function(activations[l], Z[str(l)])
            dW[str(l)] = 1/m * np.dot(dZ[str(l)], vectorization(A, architecture, l, 'forward').T)
            db[str(l)] = 1/m * np.sum(dZ[str(l)], axis=1, keepdims=True)    
        
        elif architecture[l] == 'maxpool':
            # Going backward through a maximum function, it is argmax function. So for the max slice part, it returns 0 for all-non max values
            # When we just follow a fully connected layer, we need to "devectorized", e.g. reshape the layer into the original matrix dimension
            for slice, i, j, k, o in slicing(vectorization(A, architecture, l, 'inverted'), l, architecture, filter_size, padding, stride, 'fwd'):
                v1 = i * s[l]
                v2 = i * s[l] + f[l]
                h1 = j * s[l]
                h2 = j * s[l] + f[l]

                # The way it works: we take the previous layer output A[l-1], we slice through (the same as forward prop)
                # We get the maximum number in that slice, we keep only this number and the rest should be 0
                # We call this slice therefore a mask, as it transforms the original output A[l-1] into dZ[l]
                # Once we get one mask, we add to dZ[l] which only contains 0 at the moment. We go through the next mask and we add it once again
                # And so it goes
                mask = (slice == np.max(slice))
                dA[str(l)][k, v1:v2, h1:h2, o] += mask * slice

        elif architecture[l] == 'avgpool':
            for slice, i, j, k, o in slicing(A, l, architecture, filter_size, padding, stride, 'fwd'):
                v1 = i * s[l]
                v2 = i * s[l] + f[l]
                h1 = j * s[l]
                h2 = j * s[l] + f[l]

                # The way it works: we take the previous layer output A[l-1], we slice through (the same as forward prop)
                # We get the maximum number in that slice, we keep only this number and the rest should be 0
                # We call this slice therefore a mask, as it transforms the original output A[l-1] into dZ[l]
                # Once we get one mask, we add to dZ[l] which only contains 0 at the moment. We go through the next mask and we add it once again
                # And so it goes
                dA[str(l)][k, v1:v2, h1:h2, o] = np.mean(slice) / np.dot(slice.shape)

        elif architecture[l] == 'conv':
            print("A faire")           

    return dW, db

In [ ]:
dw, db = convolutional_backward_propagation(A, Z, dA, dZ, architecture, filter_size, padding, stride)

In [ ]:
# print(k, v1, v2, h1, h2, o, dA[str(l)].shape, dA[str(l)][k, v1:v2, h1:h2, o].shape, slice.shape, A[str(l-1)].shape)
np.sum(dA['2'])

In [ ]:
W, b = update_parameters(len(architecture), W, b, dW, db, learning_rate=0.01)

print(W)
print(b)

## VI. LeNet-5 Full Model
___